In [1]:
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import tensorflow as tf
import numpy as np
#import scipy.special as sci
import matplotlib.pyplot as plt
import pickle#5 as pickle
import matplotlib as mpl
from numpy import linalg as LA

2024-08-25 22:42:39.192204: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-25 22:42:39.323344: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-25 22:42:39.348273: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 22:42:39.851292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='true'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
import seaborn as sns
sns.set_context("notebook", font_scale=2.1, rc={"lines.linewidth":2.5})
pad=20

In [4]:
#tf.compat.v1.random.set_random_seed(1234)
#tf.random.set_random_seed(0)
np.random.seed(seed=0 )

In [5]:
def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]        
    xavier_stddev = np.sqrt(2.0/(in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev,dtype=tf.float32,seed=0), dtype=tf.float32)
    
def xavier_init_small(size):
    in_dim = size[0]
    out_dim = size[1]        
    xavier_stddev = 0.01*np.sqrt(2.0/(in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev,dtype=tf.float32,seed=0), dtype=tf.float32)    
    
def RePU(inputs):
    """Rectified Power Unit Activation

    z->max(0,z)^p
    """
    return (tf.nn.relu(inputs)  )**2


def ShiftedRePU(inputs):
    """Shifted Rectified Power Unit Activation

    z->max(0,z)^p - max(0,z-0.5)^p
    """
    g=(tf.nn.relu(inputs)  )**2-(tf.nn.relu(inputs-0.5)  )**2
    return g    
       
# generate $\phi$
#Neural network for potential term $V(x)=\beta  ||x||^2 +\frac{1}{2} \sum_{i=1}^{m} (U_i(x)+\sum_{j=1}^m \gamma_{i,j}x_i)^2$ where $U_i(x)=w \phi_i(x)+b$
def neural_potential(X, weights, biases,Gamma,beta):
    num_layers = len(weights) + 1  
    H=X
    for l in range(0,num_layers-2):
        W = weights[l]
        b = biases[l]
        H = ShiftedRePU(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = 1/2*tf.reduce_sum( ( tf.matmul(X, Gamma[0])   +(tf.add(tf.matmul(H, W), b)) )**2 ,1, keepdims=True)+beta[0]*tf.reduce_sum(X**2, axis=1, keepdims=True)
    return Y


In [6]:
def compute_gradient(X, weights, biases, Gamma, beta):
    with tf.GradientTape() as tape:
        tape.watch(X)
        potential = neural_potential(X, weights, biases, Gamma, beta)
    gradient = tape.gradient(potential, X)
    return gradient

In [7]:
X = tf.cast(np.array([0.55778486, 1.70096483, 0.19261997])[None,:], dtype=tf.float32)

i=200000
open_file=open( '../saved_checkpoints/hyper100000.pkl', "rb")  
data= pickle.load(open_file)
weights=data['weights_potential']
biases=data['biases_potential']
Gamma=data['Gamma']
beta =data['beta']


grad = compute_gradient(X, weights, biases, Gamma, beta)
import tensorflow as tf
grad_numpy = grad.numpy()
print(grad_numpy)

2024-08-25 22:42:41.514302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-25 22:42:41.954418: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-08-25 22:42:41.954456: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22278 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:19:00.0, compute capability: 8.6


[[-0.15960693  0.3548584  -0.00251341]]


2024-08-25 22:42:42.447573: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
